## Feature Extraction Functions

In [1]:
from xml.dom import minidom
from collections import defaultdict 
import numpy as np
import editdistance
import pickle
import random
import time

EMBEDDING_DIM = 8
MAX_TARGET_LENGTH = 30

MAXN_STATE_NODES = 100 # maximum number of state nodes used
MAX_TOKEN_LENGTH = 60 # maximum token length padded to

NODE_KEY_LIST = [ 
    # slot names (keys) of a node to add as features
    "index", # integer
    "bounds", # interval
    "resource-id", "class", # formatted string
]
NODE_KEY_DICT = {NODE_KEY_LIST[i]:i for i in range(len(NODE_KEY_LIST))}

CHAR_LIST = ["<PAD>", "<UNK>"] +\
list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") +\
list("abcdefghijklmnopqrstuvwxyz") +\
list("0123456789") +\
list("`~!@#$%^&*()_+-={}|[]:;'',.<>/?") +\
["\\"] + ['"']
CHAR_DICT = defaultdict(
    lambda:CHAR_LIST.index("<UNK>"), 
    {CHAR_LIST[i]:i for i in range(len(CHAR_LIST))}
)

PADDING_NODE_VECTOR = [ [CHAR_DICT["<PAD>"] for _ in range(MAX_TOKEN_LENGTH)] for _ in range(len(NODE_KEY_LIST))]

## Pipeline Utils

In [2]:
class Tracker(object):
    def __init__(self):
        self.tracker = {}
        
    def add_space(self, context, space):
        # add new space to a context
        # if the context exists already, ignore the request and do nothing
        if context in self.tracker.keys():
            pass
        else:
            self.tracker[context] = space
            
    def get_space(self, context):
        return self.tracker[context]
    
    def update_space(self, context):
        # provide a context, remove it from tracker
        # and propagate the changes to all sub-contexts
        action = context[-1] # last element is the action
        prefix = context[:-1] # others are the context
        self.tracker[prefix].remove(action) # prefix should exist, otherwise exception
        if len(self.tracker[prefix]) == 0:
            # propagate if exhausted
            self.update_space(prefix)
        
    def count_space(self):
        cnt = 0
        for dkey in self.tracker.keys():
            cnt += len(self.tracker[dkey])
        return cnt

In [3]:
class Logger(object):
    def __init__(self, path, interval=10):
        self.path = path
        self.interval = interval
        self.history = []
        
    def add(self, entry):
        self.history.append(entry)
        if len(self.history) % self.interval == 0:
            with open(self.path, "wb") as f:
                pickle.dump(self.history, f)

In [4]:
def action_filter(arg_alist):
    return arg_alist
    # remove system Back/Home gui elements
#     tmp0 = [
#         arg_alist[i] 
#         for i in range(len(arg_alist)) 
#         if arg_alist[i].attributes["resource-id"] not in
#         ["com.android.systemui:id/back", "com.android.systemui:id/home", "com.android.systemui:id/menu_container"]
#     ]
#     if len(tmp0)>0:
#         return tmp0
#     else:
#         tmp1 = [
#             arg_alist[i] 
#             for i in range(len(arg_alist)) 
#             if arg_alist[i].attributes["resource-id"] not in
#             ["com.android.systemui:id/home", "com.android.systemui:id/menu_container"]
#         ]
#         if len(tmp1)>0:
#             return tmp1
#         else:
#             return arg_alist
#     tmp1 = [
#         tmp0[i] 
#         for i in range(len(tmp0)) 
#         if "android.widget.EditText" not in tmp0[i].attributes["class"]
#     ]
#     return tmp1

In [5]:
def rollout(arg_config):
    
    for ep in range(arg_config["n_episodes"]):
        print("# episode {}".format(ep))
        
        # reset
        # arg_config["environment"].clear_user_data()
        arg_config["environment"].launch_app()
        time.sleep(2)
        
        rollout_action_ids = []
        

        for i in range(arg_config["maxn_steps"]):
            current_context = tuple(rollout_action_ids)

            i_observation = arg_config["environment"].get_current_state()
            i_ids = action_filter(
                arg_config["environment"].get_available_actionable_elements(i_observation)
            )
            
            ## FIXME: This is not a real fix, we need to have a way to know whether next activity is loaded. Since,
            ## sometimes dumping window xml happens before even next activity or the screen is loaded. Now, we simply
            ## introduce a delay of 2 sec if available actions are empty and dump the window hierarchy again. However,
            ## even if i_ids still contains actions, that may be stale actions from previous activity. So, we should have
            ## a way to know that the next activity or screen is actually loaded before window xml is dumped

            if len(i_ids) == 0:
                time.sleep(2)
                i_observation = arg_config["environment"].get_current_state()
                i_ids = action_filter(
                    arg_config["environment"].get_available_actionable_elements(i_observation)
                )
            
            # tracker
            action_maskings = [0 for _ in range(len(i_ids))]
            if arg_config["tracker"] is not None:
                # print("# add space: {}, {}".format(current_context, list(range(len(i_ids)))))
                arg_config["tracker"].add_space(current_context, list(range(len(i_ids))))
                tmp_av = arg_config["tracker"].get_space(current_context)
                # print("# tmp_av: {}".format(tmp_av))
                # print("# action_maskings: {}".format(action_maskings))
                # set maskings
                for q in tmp_av:
                    action_maskings[q] = 1
            else:
                action_maskings = [1 for _ in range(len(action_maskings))]

            # explore
            selected_action_id = random.choices(list(range(len(i_ids))), weights=action_maskings, k=1)[0]
            rollout_action_ids.append(selected_action_id) # action is action_id in this case
            arg_config["environment"].perform_action(i_ids[selected_action_id])
            
            rlist = arg_config["environment"].get_reached_goal_states("train")
            rlist = [p for p in rlist if p != '<com.zoffcc.applications.aagtl.ImageManager: void DownloadFromUrl(java.lang.String,java.lang.String)> : 11'] # 4s done
            arg_config["logger"].add({
                "ts":time.time(), "run": arg_config["run_tracker"], "episode": ep, "step": i,
                "actions": len(i_ids), "selected_id": selected_action_id, "selected_action": i_ids[selected_action_id],
                "goal": rlist, 
            })
            if len(rlist)>0:
                # goal state!
                print("# rlist: {}".format(rlist))
                print("# goal state!")
                return
        
        if arg_config["tracker"] is not None:
            # mark the action sequence taken
            arg_config["tracker"].update_space(tuple(rollout_action_ids))
        print("  steps={}, actions={}".format(i, rollout_action_ids))
        print("  tracker_space={}".format(arg_config["tracker"].count_space()))

## set up environment

In [6]:
from main import *

CURR_DIR = os.path.dirname(os.getcwd())
OUTPUT_DIR = os.path.join(CURR_DIR, "results")

args = {
#     "path": "../test/com.github.cetoolbox_11/app_simple0.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/tmp/Wordpress_394/Wordpress_394.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/tmp/com.zoffcc.applications.aagtl_31/com.zoffcc.applications.aagtl_31.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/tmp/Translate/Translate.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/tmp/com.chmod0.manpages_3/com.chmod0.manpages_3.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/tmp/Book-Catalogue/Book-Catalogue.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/test/out.andFHEM.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/test/out.blue-chat.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/test/out.CallMeter3G-debug.apk",
#     "path": "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/test/out.Lucid-Browser.apk",
    "path": "../results/test_app_1/testapp_1.apk",
    "output": "../results/",
}

if args["path"] is not None:
    pyaxmlparser_apk = APK(args["path"])
    apk_base_name = os.path.splitext(os.path.basename(args["path"]))[0]

else:
    parser.print_usage()

if args["output"] is not None:
    OUTPUT_DIR = args["output"]

output_dir = os.path.join(OUTPUT_DIR, apk_base_name)

if os.path.exists(output_dir):
    rmtree(output_dir)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Setting the path for log file
log_path = os.path.join(output_dir, 'analysis.log')
log = init_logging('analyzer.%s' % apk_base_name, log_path, file_mode='w', console=True)

# Record analysis start time
now = datetime.datetime.now()
analysis_start_time = now.strftime(DATE_FORMAT)
info('Analysis started at: %s' % analysis_start_time)
start_time = time.time()

# Get the serial for the device attached to ADB
device_serial = get_device_serial(log)

if device_serial is None:
    log.warning("Device is not connected!")
    sys.exit(1)

# Initialize the uiautomator device object using the device serial
uiautomator_device = u2.connect(device_serial)
run_adb_as_root(log)
apk = Apk(args["path"], uiautomator_device, output_dir, log)
apk.launch_app()
# to track some goal state at startup, you don't have to do this
apk.clean_logcat()

[INFO] | 2021-02-25 09:13:11 AM | analyzer.testapp_1 | Adb is running with root priviledges now!


[#] Analysis started at: 2021-02-25 09:13:11 AM


[INFO] | 2021-02-25 09:13:11 AM | analyzer.testapp_1 | Old logcat messages cleared!
[INFO] | 2021-02-25 09:13:12 AM | analyzer.testapp_1 | APK installtion done for testapp_1.apk
[INFO] | 2021-02-25 09:13:12 AM | analyzer.testapp_1 | Kill the current app if already spawned!
[INFO] | 2021-02-25 09:13:12 AM | analyzer.testapp_1 | APK is already killed
[INFO] | 2021-02-25 09:13:12 AM | analyzer.testapp_1 | Spawning the current app
[INFO] | 2021-02-25 09:13:13 AM | analyzer.testapp_1 | Apk spawned successfully!
[INFO] | 2021-02-25 09:13:15 AM | analyzer.testapp_1 | Old logcat messages cleared!


In [7]:
tr = Tracker()
lg = Logger("../results/log.pkl")
config = {
    "environment": apk,
    "maxn_steps": 3,
    "n_episodes": 1000000,
    "tracker": tr,
    "logger": lg,
    "run_tracker": 0
}
start_time = time.time()
while True:
    rollout(config)
#     try:
#         rollout(config)
#     except:
#         config["run_tracker"] += 1
#         continue
    # break
end_time = time.time()
print("# total time: {}".format(end_time-start_time))

[INFO] | 2021-02-25 09:13:15 AM | analyzer.testapp_1 | Kill the current app if already spawned!


# episode 0


[INFO] | 2021-02-25 09:13:15 AM | analyzer.testapp_1 | APK killed
[INFO] | 2021-02-25 09:13:16 AM | analyzer.testapp_1 | Spawning the current app
[INFO] | 2021-02-25 09:13:16 AM | analyzer.testapp_1 | Apk spawned successfully!


IndexError: list index out of range

In [ ]:
apk.clear_user_data()

In [ ]:
i_observation = apk.get_current_state()
# apk.get_available_actionable_elements(i_observation)
action_filter(apk.get_available_actionable_elements(i_observation))

In [ ]:
apk.get_reached_goal_states("train")

In [ ]:
apk.get_available_actionable_elements(i_observation)[-2].attributes

In [ ]:
action_filter(apk.get_available_actionable_elements(i_observation))[-1].attributes

In [ ]:
apk.perform_action(
    action_filter(apk.get_available_actionable_elements(i_observation))[-6]
)

In [ ]:
tk = Tracker()

In [ ]:
tk.add_space((), [1,2,3,4])

In [ ]:
tk.add_space((1,), [11,12,13,14])

In [ ]:
tk.update_space((1,14))

In [ ]:
tk.tracker